### Silver Layer - Full Load
- Purpose: Clean and standardize data from Bronze layer
- Layer: Silver (Clean Data)
- Load Type: Full

---
### Dependencias
---

In [2]:
import os
from datetime import datetime, timedelta
import logging
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark import StorageLevel
from functools import reduce

StatementMeta(, 91c4a12e-ba54-47aa-a1d8-76f957c7657a, 4, Finished, Available, Finished)

---
### Parametros
---

In [3]:
execution_date = os.environ.get("execution_date", datetime.now().isoformat())

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

StatementMeta(, 91c4a12e-ba54-47aa-a1d8-76f957c7657a, 5, Finished, Available, Finished)

---
### Configuraciones de optimización
---

In [4]:
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")

StatementMeta(, 91c4a12e-ba54-47aa-a1d8-76f957c7657a, 6, Finished, Available, Finished)

---
### Tratamiento de la data
---

In [5]:
class SilverFullProcessor:
    def __init__(self, spark_session):
        self.spark = spark_session
        
    def get_bronze_tables(self):
        tables_df = self.spark.sql("""
            SHOW TABLES LIKE 'bronze_*'
        """)
        all_tables = [row.tableName for row in tables_df.collect()]
        
        excluded_tables = ['bronze_execution_log', 'bronze_notebook_execution_summary']
        filtered_tables = [table for table in all_tables if table not in excluded_tables]
        
        logger.info(f"Found {len(filtered_tables)} business tables to process (excluded {len(excluded_tables)} metadata tables)")
        return filtered_tables

    def clean_column_names(self, df):
        return df
    
    def standardize_data_types(self, df, table_name):
        schema_dict = dict(df.dtypes)
        transformations = []
        
        for col_name, col_type in schema_dict.items():
            new_col = col_name
            
            if col_name.lower().endswith('date') and col_type == 'bigint':
                new_col = F.when(
                    F.col(col_name) > 1000000000000,  # Si > año 2001, son nanosegundos
                    F.from_unixtime(F.col(col_name) / 1000000000).cast(TimestampType())
                ).otherwise(
                    F.from_unixtime(F.col(col_name)).cast(TimestampType())
                ).alias(col_name)
                transformations.append(new_col)
            
            elif col_name.lower() == 'month' and col_type == 'bigint':
                new_col = F.from_unixtime(F.col(col_name) / 1000000000).cast(TimestampType()).alias(col_name)
                transformations.append(new_col)
                
            elif col_name.lower() == 'dwcreateddate' and col_type != 'timestamp':
                if col_type in ['string', 'datetime']:
                    new_col = F.to_timestamp(F.col(col_name)).alias(col_name)
                    transformations.append(new_col)
                else:
                    transformations.append(F.col(col_name))
            
            elif col_name.lower().endswith(('_key', 'key')) and col_type == 'string':
                new_col = F.when(
                    F.trim(F.upper(F.col(col_name))).isin("", "NULL", "N/A", "UNKNOWN", "NONE") |
                    F.trim(F.col(col_name)).isNull(),
                    None
                ).otherwise(
                    F.trim(F.upper(F.col(col_name))) 
                ).alias(col_name)
                transformations.append(new_col)
            
            elif col_type == 'string':
                new_col = F.when(
                    F.trim(F.upper(F.col(col_name))).isin("", "NULL", "N/A", "UNKNOWN", "NONE", "#N/A") |
                    F.trim(F.col(col_name)).isNull(),
                    None
                ).otherwise(
                    F.trim(F.col(col_name))
                ).alias(col_name)
                transformations.append(new_col)
            
            elif col_type in ['double', 'float']:
                new_col = F.when(
                    F.isnan(F.col(col_name)) | F.col(col_name).isNull(), None
                ).otherwise(F.col(col_name)).alias(col_name)
                transformations.append(new_col)
            
            elif col_type == 'boolean':
                transformations.append(F.col(col_name))
                
            else:
                transformations.append(F.col(col_name))
        
        if transformations:
            df = df.select(*transformations)
        return df

    def create_quarantine_records(self, df, table_name):                
        problem_filter = None
        
        future_cutoff = F.date_add(F.current_date(), 730)
        for col_name in df.columns:
            if col_name.lower().endswith('date') and col_name.lower() != 'dwcreateddate':
                date_condition = F.col(col_name) > future_cutoff
                problem_filter = date_condition if problem_filter is None else (problem_filter | date_condition)
        
        for col_name, col_type in dict(df.dtypes).items():
            if col_type in ['double', 'float'] and 'value' in col_name.lower():
                value_condition = (F.col(col_name) > 100000000) | (F.col(col_name) < -10000000)
                problem_filter = value_condition if problem_filter is None else (problem_filter | value_condition)
        
        if problem_filter is not None:
            try:
                quarantine_df = df.filter(problem_filter)
                clean_df = df.filter(~problem_filter)
                
                if quarantine_df.count() > 0:
                    quarantine_table = f"silver_quarantine_{table_name.replace('silver_', '')}"
                    
                    quarantine_with_metadata = (
                        quarantine_df
                        .withColumn("quarantine_date", F.current_timestamp())
                        .withColumn("quarantine_reason", F.lit("data_quality_issues"))
                        .withColumn("source_table", F.lit(table_name))
                    )
                    
                    quarantine_with_metadata.write.mode("append").format("delta").saveAsTable(quarantine_table)
                    logger.info(f"Quarantined {quarantine_df.count()} problematic records from {table_name}")
                
                return clean_df
            except Exception as e:
                logger.warning(f"Quarantine process failed for {table_name}: {str(e)}. Returning original data.")
                return df
        return df    

    def optimize_partitioning(self, df, table_name):
        row_count = df.count()
        current_partitions = df.rdd.getNumPartitions()
        
        if row_count > 1000000: 
            base_partitions = row_count // 150000
            if base_partitions < 4:
                optimal_partitions = 4
            elif base_partitions > 200:
                optimal_partitions = 200
            else:
                optimal_partitions = base_partitions
        else:  
            calculated_partitions = row_count // 50000
            optimal_partitions = 1 if calculated_partitions < 1 else calculated_partitions
        
        if current_partitions != optimal_partitions:
            logger.info(f"{table_name}: Repartitioning from {current_partitions} to {optimal_partitions}")
            df = df.repartition(optimal_partitions)
        return df
    
    def remove_duplicates(self, df, table_name):
        if table_name.startswith('silver_dim_'):
            cols_except_created = [c for c in df.columns if c.lower() != 'dwcreateddate']
            if 'dwcreateddate' in df.columns:
                window_spec = Window.partitionBy(*cols_except_created).orderBy(F.desc('dwcreateddate'))
                df = (df.withColumn('rn', F.row_number().over(window_spec))
                        .filter(F.col('rn') == 1)
                        .drop('rn'))
        elif table_name.startswith('silver_fact_'):
            key_columns = []
            for col_name in df.columns:
                if any(pattern in col_name.lower() for pattern in ['_number', '_key', 'customer_key', 'product_key']):
                    key_columns.append(col_name)
            if key_columns and 'dwcreateddate' in df.columns:
                window_spec = Window.partitionBy(*key_columns).orderBy(F.desc('dwcreateddate'))
                df = (df.withColumn('rn', F.row_number().over(window_spec))
                        .filter(F.col('rn') == 1)
                        .drop('rn'))
        return df

    def data_quality_checks(self, df, table_name):
        initial_count = df.count()
        df_clean = df.dropna(how='all')
        df_clean = self.create_quarantine_records(df_clean, table_name)
        final_count = df_clean.count()
        processed_rows = initial_count - final_count
        if processed_rows > 0 and initial_count > 0:
            logger.info(f"{table_name}: Processed {processed_rows} problematic rows ({(processed_rows/initial_count)*100:.2f}%)")
        return df_clean
    
    def process_silver_table(self, bronze_table_name):
        try:
            silver_table_name = bronze_table_name.replace('bronze_', 'silver_')
            logger.info(f"Processing {bronze_table_name} -> {silver_table_name}")
            bronze_df = self.spark.table(bronze_table_name)
            
            if bronze_df.count() == 0:
                logger.warning(f"Bronze table {bronze_table_name} is empty")
                return 0
            
            df_processed = bronze_df
            df_processed = self.standardize_data_types(df_processed, silver_table_name)
            df_processed = self.remove_duplicates(df_processed, silver_table_name)
            df_processed = self.data_quality_checks(df_processed, silver_table_name)
            df_processed = self.optimize_partitioning(df_processed, silver_table_name)
            
            df_processed = (df_processed
                .withColumn('silver_created_date', F.current_timestamp())
                .withColumn('silver_execution_id', F.lit(execution_date))
            )
            
            df_processed = df_processed.persist(StorageLevel.MEMORY_AND_DISK)
            record_count = df_processed.count()
            writer = df_processed.write.mode("overwrite").format("delta")
            
            if 'fact' in silver_table_name:
                writer = (writer
                          .option("optimizeWrite", "true")
                          .option("autoCompact", "true")
                          .option("dataSkippingNumIndexedCols", "5"))
            else:
                writer = writer.option("optimizeWrite", "true")
            
            writer.saveAsTable(silver_table_name)
            logger.info(f"Final partitions for {silver_table_name}: {df_processed.rdd.getNumPartitions()}")            
            df_processed.unpersist()
            logger.info(f"Successfully processed {silver_table_name}: {record_count:,} records")
            return record_count
            
        except Exception as e:
            logger.error(f"Error processing {bronze_table_name}: {str(e)}")
            raise e


StatementMeta(, 91c4a12e-ba54-47aa-a1d8-76f957c7657a, 7, Finished, Available, Finished)

In [6]:
silver_processor = SilverFullProcessor(spark)

StatementMeta(, 91c4a12e-ba54-47aa-a1d8-76f957c7657a, 8, Finished, Available, Finished)

In [7]:
bronze_tables = silver_processor.get_bronze_tables()
logger.info(f"Found {len(bronze_tables)} bronze tables to process")

results = []
total_records = 0

for bronze_table in bronze_tables:
    try:
        record_count = silver_processor.process_silver_table(bronze_table)
        silver_table = bronze_table.replace('bronze_', 'silver_')
        
        results.append({
            'bronze_table': bronze_table,
            'silver_table': silver_table,
            'record_count': record_count,
            'status': 'success'
        })
        total_records += record_count
        print(f"✓ {silver_table}: {record_count:,} records")
        
    except Exception as e:
        results.append({
            'bronze_table': bronze_table,
            'silver_table': bronze_table.replace('bronze_', 'silver_'),
            'record_count': 0,
            'status': 'failed',
            'error': str(e)
        })
        print(f"✗ {bronze_table}: FAILED - {str(e)}")
    
    print("-" * 50)

StatementMeta(, 91c4a12e-ba54-47aa-a1d8-76f957c7657a, 9, Finished, Available, Finished)

INFO:__main__:Found 15 business tables to process (excluded 2 metadata tables)
INFO:__main__:Found 15 bronze tables to process
INFO:__main__:Processing bronze_dim_brands -> silver_dim_brands
INFO:__main__:silver_dim_exchange_rate: Repartitioning from 2 to 1
INFO:__main__:silver_dim_products: Repartitioning from 8 to 5
INFO:__main__:silver_fact_budget: Repartitioning from 6 to 5
INFO:__main__:silver_fact_invoices: Repartitioning from 20 to 113
INFO:__main__:silver_fact_orders: Processed 1048314 problematic rows (6.63%)
INFO:__main__:silver_fact_orders: Repartitioning from 15 to 98
INFO:__main__:silver_incremental_control: Repartitioning from 6 to 1


✓ silver_dim_brands: 20 records
--------------------------------------------------
✓ silver_dim_budget_rate: 15 records
--------------------------------------------------
✓ silver_dim_customers: 3,911 records
--------------------------------------------------
✓ silver_dim_employees: 893 records
--------------------------------------------------
✓ silver_dim_exchange_rate: 57,900 records
--------------------------------------------------
✓ silver_dim_invoice_doctype: 5 records
--------------------------------------------------
✓ silver_dim_order_doctype: 4 records
--------------------------------------------------
✓ silver_dim_order_status: 6 records
--------------------------------------------------
✓ silver_dim_products: 256,293 records
--------------------------------------------------
✓ silver_dim_regions: 181 records
--------------------------------------------------
✓ silver_fact_budget: 276,151 records
--------------------------------------------------
✓ silver_fact_forecast: 5,1

---
### Logs del proceso
---

In [8]:
execution_log_data = [(
    execution_date,
    "silver_full_load",
    datetime.now(),
    "completed" if all(r["status"] == "success" for r in results) else "completed_with_errors",
    "silver",
    "full",
    total_records,
    len([r for r in results if r["status"] == "success"]),
    len([r for r in results if r["status"] == "failed"]),
    str(results)[:1000]
)]

execution_log_schema = StructType([
    StructField("execution_id", StringType(), True),
    StructField("pipeline_name", StringType(), True),
    StructField("execution_timestamp", TimestampType(), True),
    StructField("status", StringType(), True),
    StructField("layer", StringType(), True),
    StructField("load_type", StringType(), True),
    StructField("total_records", LongType(), True),
    StructField("successful_tables", IntegerType(), True),
    StructField("failed_tables", IntegerType(), True),
    StructField("details", StringType(), True)
])

execution_log = spark.createDataFrame(execution_log_data, execution_log_schema)
execution_log.write.format("delta").mode("append").saveAsTable("silver_execution_log")

StatementMeta(, 91c4a12e-ba54-47aa-a1d8-76f957c7657a, 10, Finished, Available, Finished)

---
### Resumen del proceso
---

In [9]:
successful_loads = len([r for r in results if r["status"] == "success"])
failed_loads = len([r for r in results if r["status"] == "failed"])

print("=" * 60)
print("SILVER FULL LOAD SUMMARY:")
print(f"Successful transformations: {successful_loads}")
print(f"Failed transformations: {failed_loads}")
print(f"Total records processed: {total_records:,}")
print(f"Execution Date: {execution_date}")
print("=" * 60)

StatementMeta(, 91c4a12e-ba54-47aa-a1d8-76f957c7657a, 11, Finished, Available, Finished)

SILVER FULL LOAD SUMMARY:
Successful transformations: 15
Failed transformations: 0
Total records processed: 32,404,976
Execution Date: 2025-09-19T19:07:22.846127


---
### Optimización
---

In [10]:
if successful_loads > 0:
    print("\nSuccessfully created silver tables:")
    for result in results:
        if result["status"] == "success":
            print(f"  - {result['silver_table']}: {result['record_count']:,} records")

print("\nOptimizing silver tables...")
for result in results:
    if result["status"] == "success":
        try:
            spark.sql(f"OPTIMIZE {result['silver_table']}")
            print(f"✓ Optimized: {result['silver_table']}")
        except Exception as e:
            print(f"✗ Error optimizing {result['silver_table']}: {e}")

print("Silver full load process completed.")

StatementMeta(, 91c4a12e-ba54-47aa-a1d8-76f957c7657a, 12, Finished, Available, Finished)


Successfully created silver tables:
  - silver_dim_brands: 20 records
  - silver_dim_budget_rate: 15 records
  - silver_dim_customers: 3,911 records
  - silver_dim_employees: 893 records
  - silver_dim_exchange_rate: 57,900 records
  - silver_dim_invoice_doctype: 5 records
  - silver_dim_order_doctype: 4 records
  - silver_dim_order_status: 6 records
  - silver_dim_products: 256,293 records
  - silver_dim_regions: 181 records
  - silver_fact_budget: 276,151 records
  - silver_fact_forecast: 5,197 records
  - silver_fact_invoices: 17,037,850 records
  - silver_fact_orders: 14,766,489 records
  - silver_incremental_control: 61 records

Optimizing silver tables...
✓ Optimized: silver_dim_brands
✓ Optimized: silver_dim_budget_rate
✓ Optimized: silver_dim_customers
✓ Optimized: silver_dim_employees
✓ Optimized: silver_dim_exchange_rate
✓ Optimized: silver_dim_invoice_doctype
✓ Optimized: silver_dim_order_doctype
✓ Optimized: silver_dim_order_status
✓ Optimized: silver_dim_products
✓ Optimi